In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import train_test_split

# import StandardScaler to perform scaling
from sklearn.preprocessing import StandardScaler

# resample the dataset
from sklearn.utils import resample
from sklearn.utils import shuffle

# import various functions from sklearn
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import StackingClassifier


# import the XGBoost function for classification
from xgboost import XGBClassifier

import random

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [4]:
data = pd.read_csv('/content/drive/MyDrive/CAPSTONE/CAPSTONE_PROJECT/08_MODELS/Model6_DamageExtent/model_6_master_data.csv')

In [9]:
df=data.copy()
df.head()

,Road Name,Cross-Street Name,Driver Substance Abuse,Driver At Fault,Injury Severity,Driver Distracted By,Drivers License State,Vehicle Body Type,Speed Limit,Driverless Vehicle,Parked Vehicle,Vehicle Year,Vehicle Make,Latitude,Longitude,Vehicle Model,hour,Crash_year,Crash_month,Crash_day,Crash_hour,Crash_week,Agency Name_Maryland_National_Capital,Agency Name_Montgomery_County_Police,Agency Name_Rockville_Police_Department,Agency Name_Takoma_Park_Police_Department,ACRS Report Type_Injury Crash,ACRS Report Type_Property Damage Crash,Route Type_County_Route,Route Type_Crossover,Route Type_Government_Route,Route Type_Interstate_Route,Route Type_Local_Route,Route Type_Maryland_State_Route,Route Type_Municipality_Route,Route Type_Other_Public_Roadway,Route Type_Private_Route,Route Type_Ramp_Route,Route Type_Service_Road,Route Type_Spur_Route,Route Type_US_State_Route,Collision Type_Angle + Left Turn,Collision Type_Angle + Right Turn,Collision Type_Angle Collision,Collision Type_Back-to-Back,Collision Type_Both Left Turns (Opposite Direction),Collision Type_Both Left Turns (Same Direction),Collision Type_Head-On,Collision Type_Head-On + Left Turn,Collision Type_Left Turn (Same Direction),Collision Type_Other,Collision Type_Rear vs Side,Collision Type_Rear-End (Same Direction),Collision Type_Right Turn (Same Direction),Collision Type_Sideswipe (Opposite Direction),Collision Type_Sideswipe (Same Direction),Collision Type_Single Vehicle,Circumstance_Category_Congestion / Backup,Circumstance_Category_Driver Inattention,Circumstance_Category_Lane Violation,Circumstance_Category_Mechanical / Environmental,Circumstance_Category_Passing / Turning,Circumstance_Category_Right-of-Way Violation,Circumstance_Category_Road Obstruction,Circumstance_Category_Speed Related,Circumstance_Category_Surface Condition,Circumstance_Category_Tailgating,Circumstance_Category_Traffic Control Violation,Circumstance_Category_Weather Related,Weather_Blowing Snow,Weather_Clear,Weather_Cloudy,Weather_Fog,Weather_Freezing Rain,Weather_Other,Weather_Rain,Weather_Severe Crosswinds,Weather_Severe Winds,Weather_Sleet,Weather_Snow,Weather_Wintry Mix,Surface Condition_Ice,Surface Condition_Mud_Dirt_Gravel,Surface Condition_Oil,Surface Condition_Other,Surface Condition_Sand,Surface Condition_Slush,Surface Condition_Snow,Surface Condition_Wet,Light_Dark - Not Lighted,Light_Dark - Unknown Lighting,Light_Dawn,Light_Daylight,Light_Dusk,Light_Other,Traffic Control_Flashing Traffic Signal,Traffic Control_Lane Use Control,Traffic Control_No Control,Traffic Control_Other,Traffic Control_Other Pavement Marking,Traffic Control_Other Signal,Traffic Control_Pedestrian Crossing,Traffic Control_Person Control,Traffic Control_Railroad Crossing Device,Traffic Control_Railroad Crossing Signal,Traffic Control_Ramp Meter Signal,Traffic Control_School Zone Sign,Traffic Control_Stop Sign,Traffic Control_Traffic Signal,Traffic Control_Warning Sign,Traffic Control_Yield Sign,Vehicle First Impact Location_Eight O Clock,Vehicle First Impact Location_Eleven O Clock,Vehicle First Impact Location_Five O Clock,Vehicle First Impact Location_Four O Clock,Vehicle First Impact Location_Nine O Clock,Vehicle First Impact Location_NonCollision,Vehicle First Impact Location_One O Clock,Vehicle First Impact Location_RoofTop,Vehicle First Impact Location_Seven O Clock,Vehicle First Impact Location_Six O Clock,Vehicle First Impact Location_Ten O Clock,Vehicle First Impact Location_Three O Clock,Vehicle First Impact Location_Twelve O Clock,Vehicle First Impact Location_Two O Clock,Vehicle First Impact Location_Underside,Vehicle First Impact Location_VehicleNotAtScene,Vehicle Movement_Backing,Vehicle Movement_ChangingLanes,Vehicle Movement_DriverlessMovingVehicle,Vehicle Movement_EnteringTrafficLane,Vehicle Movement_LeavingTrafficLane,Vehicle Movement_MakingLeftTurn,Vehicle Movement_MakingRightTurn,Vehicle Movement_MakingUTurn,Vehicle Movement_MovingConstantSpeed,Vehicle Movement_NegotiatingCurve,Vehicle Movement_O

In [10]:
df['Damage_Class'].value_counts()

,count
Damage_Class,
1,72161
2,50027
5,47723
0,6867
3,5968
6,2145
4,94


In [11]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

x = df.drop(columns='Damage_Class')
y = df['Damage_Class'].value_counts()

classes = np.unique(y)

class_weights = compute_class_weight(
    class_weight="balanced",
    classes=classes,
    y=y
)

class_weight_dict = {cls: weight for cls, weight in zip(classes, class_weights)}

print(class_weight_dict)


{np.int64(94): np.float64(1.0), np.int64(2145): np.float64(1.0), np.int64(5968): np.float64(1.0), np.int64(6867): np.float64(1.0), np.int64(47723): np.float64(1.0), np.int64(50027): np.float64(1.0), np.int64(72161): np.float64(1.0)}


## 📌 Why Class Weight is Used Instead of SMOTE

The dataset has **extreme class imbalance**, especially:
- Large classes: 40k–70k samples  
- Class 4: **only 94 samples**

### ❗ Why SMOTE is avoided
- SMOTE needs enough samples to create valid synthetic points.  
- With only 94 samples, it produces **unrealistic and noisy data**.  
- This leads to **overfitting** and distorted feature space.

### ✔ Why Class Weight is better
- No fake data is created.  
- Model simply **penalizes mistakes on minority classes more**.  
- Works reliably even when a class has **very few samples**.

### ✅ Conclusion
Because Class 4 is extremely small, **SMOTE is unsafe**, while **class weights balance the learning without damaging data quality**.  
Hence, **class weight is the preferred method** for this dataset.
